### Intalling libs

In [ ]:
# ! pip install --upgrade rfpimp
# ! pip install --upgrade scikit-learn
# ! pip install --upgrade pyarrow
# ! pip install --upgrade yellowbrick
# ! pip install --upgrade pandas

### Column description

| Variável 	| Descrição 	|
|:-:	|:-	|
| PassangerID 	| ID de identificação do passageiro(a) 	|
| Survived 	| se o passageiro(a) sobreviveu (0 = não, 1 = sim) 	|
| Pclass 	| classe do passageiro:<br>     * **1 = primeira**,<br>     * **2 = segunda**,<br>     * **3 = terceira** 	|
| name 	| nome do passageiro(a) 	|
| sex 	| sexo do passageiro(a) 	|
| age 	| idade do passageiro(a) 	|
| Sibsp 	| número de irmão(ãs)/esposo(a) à bordo 	|
| Parch 	| número de pais/filhos(as) à bordo 	|
| Ticket 	| número da passagem 	|
| Fare 	| preço da passagem 	|
| Cabin 	| cabine 	|
| Embarked 	| local que o passageiro(a) embarcou:<br>     * **C = Cherboug**,<br>     * **Q = Queenstown**,<br>     * **S = Southamption** 	|
| WikiId 	| ID de identificação do passageiro(a) segundo Wikipedia 	|
| Name_wiki 	| nome do passageiro(a) 	|
| Age_wiki 	| idade do passageiro(a) 	|
| Hometown 	| cidade de nascimento do passageiro(a) 	|
| Boarded 	| cidade de embarque 	|
| Destination 	| destino da viagem 	|
| Lifeboat 	| identificação do bote salva-vidas 	|
| Body 	| número de identificação do corpo 	|


<font color='red'>**IMPORTANT**</font>

The new features (the ones after 'Embarked') are very similar to the original ones but they are more up-to-date and have much fewer missing values. Therefore, users can decide on the preferred features themselves.

### Importing Libs

In [ ]:
# data visualization
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from seaborn import (
    jointplot,
    pairplot,
    boxplot,
    heatmap
)
from yellowbrick.features import (
    JointPlotVisualizer,
    Rank2D, 
    RadViz,
    ParallelCoordinates
)

# data manipulation
import numpy as np
import pandas as pd
from pandas.plotting import(
    radviz
)
import janitor as jn
from ydata_profiling import ProfileReport

# missing values
import missingno as msno

from sklearn.impute import (
    SimpleImputer
)

# machine learning models
from sklearn import (
    ensemble,
    preprocessing,
    tree,
    impute,
    model_selection
)

from sklearn.dummy import (
    DummyClassifier
)

from sklearn.model_selection import (
    train_test_split
)

from sklearn.experimental import (
    enable_iterative_imputer
)

from sklearn.linear_model import (
    LogisticRegression
)

from sklearn.tree import (
    DecisionTreeClassifier
)

from sklearn.neighbors import (
    KNeighborsClassifier
)

from sklearn.naive_bayes import (
    GaussianNB
)

from sklearn.svm import (
    SVC
)

from sklearn.ensemble import (
    RandomForestClassifier
)

import xgboost

# data model metrics
from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    precision_score,
    recall_score
)

from yellowbrick.classifier import (
    ConfusionMatrix
)

from yellowbrick.model_selection import (
    LearningCurve
)

# data prep-model
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    learning_curve
)

# model deploy
import pickle

### Reading the Titanic Dataset

In [ ]:
df = pd.read_csv("titanic_dataset.csv", index_col=0)
df.head(5)

### Deleting _Class_ feature at the end
We are deleting because is the same as _pclass_ (same result, same data)

In [ ]:
df = df.drop('Class', axis = 'columns')
df

### Converting DataFrame Column Names to Lowercase snakecase

In [ ]:
df.columns = (df.columns
                .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
                .str.lower()
             )

df.dtypes

### Preparing the dataset

In [ ]:
'''
# dropping columns taht do not add value
df_heatmap_collinearity = df.drop(columns = ['name',
                        'name_wiki',
                        'wiki_id',
                        'hometown',
                        'destination',
                        'ticket',
                        'lifeboat',
                        'body',
                        'age'])

# using get_dummies function to convert object to int
df_heatmap_collinearity = pd.get_dummies(df)

# dropping redundant features
df_heatmap_collinearity = df_heatmap_collinearity.drop(columns = ['sex_male'])

#remove rows with any values that are not finite (NaN or infite)
df_heatmap_collinearity = df_heatmap_collinearity[np.isfinite(df_heatmap_collinearity).all(1)]

# first, we need to create a series of the target feature
y = df_heatmap_collinearity.survived

# then, we create a DataFrame with the attributes
X = df_heatmap_collinearity.drop(columns = ['survived'])

# using the scikit-learn to split 30% to test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42'
'''

In [ ]:
'''
df_fea_eng = df.drop(columns = ['name',
'ticket',
'sex',
#'cabin',
'embarked',
'name_wiki',
'hometown',
'boarded',
'destination',
'lifeboat',
'body']'
'''

### Feature Engineering
We use feature selection to select variables that are useful for the model. Irrelevant attributes can have a negative effect on our models.

In [ ]:
def tweak_titanic(df):
    df = df.drop(
        columns=[
            "name",
            "ticket",
            "body",
            "cabin",
        ]
    ).pipe(pd.get_dummies, drop_first=True)
    return df

def get_train_test_X_y(
    df, y_col, size=0.3, std_cols=None
):
    y = df[y_col]
    X = df.drop(columns=y_col)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=size, random_state=42
    )
    cols = X.columns
    num_cols = [
        "pclass",
        "age_wiki",
        "sib_sp",
        "parch",
        "fare",
    ]
    fi = impute.IterativeImputer()
    fitted = fi.fit_transform(X_train[num_cols])
    X_train = X_train.assign(**{c:fitted[:,i] for i, c in enumerate(num_cols)})
    test_fit = fi.transform(X_test[num_cols])
    X_test = X_test.assign(**{c:test_fit[:,i] for i, c in enumerate(num_cols)})
    if std_cols:
        std = preprocessing.StandardScaler()
        fitted = std.fit_transform(X_train[std_cols])
        X_train = X_train.assign(**{c:fitted[:,i] for i, c in enumerate(std_cols)})
        test_fit = std.transform(X_test[std_cols])
        X_test = X_test.assign(**{c:test_fit[:,i] for i, c in enumerate(std_cols)})

    return X_train, X_test, y_train, y_test

orig_df = df

ti_df = tweak_titanic(orig_df)
std_cols = "pclass,age_wiki,sib_sp,fare".split(",")
X_train, X_test, y_train, y_test = get_train_test_X_y(
    ti_df, "survived", std_cols=std_cols
)

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [ ]:
import rfpimp
rfpimp.plot_dependence_heatmap(
    rfpimp.feature_dependence_matrix(X_train),
    value_fontsize=12,
    label_fontsize=14,
    figsize=(8, 8)
)
fig = plt.gcf()

In [ ]:
# cabin feature
agg = (
    df_fea_eng.groupby("cabin")
    .agg("min,max,mean,sum".split(","))
    .reset_index()
)

agg.columns = [
    "_".join(c).strip("_")
    for c in agg.columns.values
]

agg_df = df_fea_eng.merge(agg, on = "cabin")

agg_df

In [ ]:
agg_df.dtypes

#### Collinear columns
We can define a collinearity limit and run code that will return features above that limit

In [ ]:
agg_df = agg_df.drop(columns = ['cabin'])

limit = 0.95
corr = agg_df.corr()

mask = np.triu(
    np.ones(corr.shape), k = 1
).astype(bool)

corr_no_diag = corr.where(mask)

coll = [
    c
    for c in corr_no_diag.columns
    if any(abs(corr_no_diag[c]) > limit)
]

coll

### Recursive attribute elimination

In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)